# Choropleth map of number of deaths from floods and per capita GDP

This notebook shows how to ploat a choropleth map using Plotly library, depicting the number of deaths from floods in the 21st century combined with the per capita GDP of the countries involved. Data is captured from Wikipedia and https://countrycode.org.

In [ ]:
"""
Install the necessary modules to run this notebook
IMPORTANT: Run this cell only once
"""

from IPython.display import display, HTML, display_html
! pip install "lxml";
! pip install "plotly";

display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True) # restart kernel

In [26]:
import pandas as pd
import numpy as np
from functools import reduce
import re
import matplotlib.pyplot as plt
import io
import requests
import plotly.graph_objects as go
from datetime import date

"""
BRIEF EXPLANATION ABOUT HOW THIS NOTEBOOK PLOTS THE CHART

STEP 1 - Open and treat dataset from wikipedia about Floods victms (Source Wikipedia online)
STEP 2 - Open and treat a dataset from CountryCode with Country codes and GDP data
STEP 3 - Perform a inner join on both datasets on country codes (3-digits)
STEP 4 - Plot a choropleth map using Plotly library, depicting GDP per capita and Deaths related to Floods

"""

        
"""
STEP 1 - Open and treat dataset from wikipedia about Floods victms (Source Wikipedia online)
"""
df_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_deadliest_floods')

''' Treat column adjusting to numerical format '''
def adjust_data_format(column):
    column_a = column.astype(str).replace('\[.*\]', '', regex=True).replace(',','', regex=True).replace(r'\+', '', regex=True)
    return column_a
    
''' Treat country names accross multiple rows '''
def split_countries(df,target_column,separator=','):
    import itertools
    df['Number_Countries'] = 1
    def split_rows(row,row_accumulator,target_column,separator):
        
        if not pd.isnull(row[target_column]):
            split_row = row[target_column].split(separator)
            for s in split_row:
                new_row = row.to_dict()
                new_row[target_column] = s.strip()
                new_row['Number_Countries'] = len(split_row)
                row_accumulator.append(new_row)
        else:
                row_accumulator.append(row)
    new_rows = []
    df.apply(split_rows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df


max_year = np.max(df_list[1].Year) # get last year from data source

del df_list[0] # drop not used table (just some introduction)
del df_list[0] # drop not used table (list of the deadliest floods)

years = list(range(max_year, 1999, -1))

df_list = df_list[:len(years)]

# index each table by year
for df, year in zip(df_list, years ):
    df.columns = ['Date', 'Death toll', 'Location', 'Event']
    df['Year'] = year
    df['Death toll'] = adjust_data_format(df['Death toll'])
    
# append all tables into a single dataframe
df_floods_deaths = pd.concat(df_list[:len(years)])
df_floods_deaths = df_floods_deaths.reset_index(drop=True)
df_floods_deaths = df_floods_deaths[df_floods_deaths['Location'].str.strip() != 'Location']

# treat Death toll column
df_floods_deaths['Death toll'] = df_floods_deaths['Death toll'].astype(int)

# treat Location column (remove cities keep country)
df_floods_deaths['Location'] = df_floods_deaths['Location'].str.replace(' Bab El Oued', '')
df_floods_deaths['Location'] = df_floods_deaths['Location'].str.replace('Jiangsu', '')
df_floods_deaths['Location'] = df_floods_deaths['Location'].str.replace('São Paulo', '')
df_floods_deaths['Location'] = df_floods_deaths['Location'].str.replace('Shaanxi Hebei', '')
df_floods_deaths['Location'] = df_floods_deaths['Location'].str.replace('Zhejiang', '')
df_floods_deaths['Location'] = df_floods_deaths['Location'].str.replace('Sumatra', '')
df_floods_deaths['Location'] = df_floods_deaths['Location'].str.replace('Maharastra', '')
df_floods_deaths['Location'] = df_floods_deaths['Location'].str.replace('Republic of Macedonia', 'Macedonia')

df_floods_deaths['Location'] = df_floods_deaths['Location'].str.strip()
df_floods_deaths['Countries affected'] = df_floods_deaths['Location']

# split countries related to the same event
df_floods_deaths = split_countries(df_floods_deaths, 'Location' )

# estimate the death toll per country (assumption: linearly distributed among involved countries)
df_floods_deaths['Death toll Estimated per Country'] = df_floods_deaths['Death toll'] / df_floods_deaths['Number_Countries']
df_floods_deaths['Death toll Estimated per Country'] = df_floods_deaths['Death toll Estimated per Country'].astype(int)

# select data only 21st century
df_floods_deaths = df_floods_deaths[df_floods_deaths['Year']>2000]

# summarize data per country and sort descending
df_floods_deaths_summarized = df_floods_deaths.groupby('Location').agg({'Death toll Estimated per Country' : 'sum'}).dropna()
df_floods_deaths_summarized = df_floods_deaths_summarized.sort_values(by='Death toll Estimated per Country', ascending=False)

"""
STEP 2 - Open and treat a dataset from CountryCode with Country codes and GDP data
"""

# reads the html
countries_data = pd.read_html('https://countrycode.org')[0]

# grab 3-digit iso-code
countries_data['CODE3'] = countries_data['ISO CODES'].str.replace('[A-Z][A-Z] / ', '', regex=True)

# treat GDP data (the source presents it in a text format, with milions, billions, etc)
countries_data['UnitGDP'] = 1.0
countries_data = countries_data[  countries_data['GDP $USD'].notnull()  ]
countries_data['UnitGDP'] = countries_data['GDP $USD'].apply(lambda x : 1000000.0 if 'Mil' in x else (1000000000.0 if 'Bil' in x else (1000000000000.0 if 'Tri' in x else 1.0 )) )
countries_data['GDP $USD'] = countries_data['GDP $USD'].str.replace(' Million', '')
countries_data['GDP $USD'] = countries_data['GDP $USD'].str.replace(' Billion', '')
countries_data['GDP $USD'] = countries_data['GDP $USD'].str.replace(' Trillion', '')
countries_data['GDP $USD'] = countries_data['GDP $USD'].astype(float)
countries_data['GDP_FULL_NUMBER'] = countries_data['UnitGDP'] * countries_data['GDP $USD']

# calculated GDP per capita
countries_data['POPULATION'] = countries_data['POPULATION'].astype(float)
countries_data['GDP_PER_CAPITA'] = countries_data['GDP_FULL_NUMBER'] / countries_data['POPULATION']


"""
STEP 3 - Perform a inner join on both datasets on country codes (3-digits"
"""
df_floods_geo = df_floods_deaths_summarized.merge(countries_data, how='inner', left_index=True, right_on='COUNTRY')
df_floods_geo = df_floods_geo.sort_values(by='Death toll Estimated per Country', ascending=False)
df_floods_geo['Rank'] = df_floods_geo['Death toll Estimated per Country'].rank(ascending=False)

# adds a summary to each record (this will be used in the next step)
df_floods_geo['SUMMARY'] = (
    '<b>' 
    + 'Country: '
    + df_floods_geo['COUNTRY']
    + '</b>'
    + '<br><br>'
    + 'Floods death toll (2001-' + str(max_year) +'): '
    + df_floods_geo['Death toll Estimated per Country'].astype(str)
    + '<br>'
    + 'Rank: top '
    + df_floods_geo['Rank'].astype(int).astype(str)
    + ' in deaths<br>'
    + '<br>'
    + 'GDP per capita (USD): '
    + round(df_floods_geo['GDP_PER_CAPITA'], 2).astype(str)
    )

"""
STEP 4 - Plot a choropleth map using Plotly library, depicting GDP per capita and Deaths related to Floods
My main reference to this code is: https://sharkcoder.com/data-visualization/plotly-maps
"""

# Embed 2 plots in the same figure
fig = go.Figure(
    data=
                    [   # GDP data
                        go.Choropleth( 
                                    locations = countries_data['CODE3'],
                                    z = countries_data['GDP_PER_CAPITA'],
                                    text = "GDP per capita (USD) of " + countries_data['COUNTRY'],
                                    colorscale = 'Blues',
                                    autocolorscale=False,
                                    reversescale=False,
                                    marker_line_color='darkgray',
                                    marker_line_width=0.5,
                                    colorbar_title = 'GDP per <br>capita (USD)',
                                    zmin = 100,
                                    zmax = 100000,  # I've limitated some countries which are outliers
                                    hoverinfo='skip'


                                 ) , 
                        # Floods data
                        go.Scattergeo(
                                            locations = df_floods_geo['CODE3'],
                                            mode='markers',
                                            marker=dict(
                                                size=df_floods_geo['Death toll Estimated per Country'],
                                                sizemode='area',
                                                # this formula is a algorithm provided by plotly to set optimal bubble size
                                                # https://plotly.com/python/bubble-charts/
                                                sizeref= ( max(df_floods_geo['Death toll Estimated per Country']) * 2 ) / (70.**2),
                                                sizemin=1,
                                                line=None,
                                                color='rgba(227,18,18,0.6)' # red
                                               ),
                                            text = df_floods_geo['SUMMARY'],
                                            hoverlabel = dict(bgcolor='white'),
                                            hovertemplate = '%{text} <extra></extra>',

                            
                                )

                    ]
               
               )

# Embelishments to the chart
fig.update_layout(
    width=1000,
    height=620,
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular',
        lataxis = dict( range = [-60, 90]), #Clip Antarctica
    ),
    title={
        'text': '<b>World GDP per capita and victims of floods during 21st century</b>',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    title_font_color='#525252',
    title_font_size=18,
    font=dict(
        family='Arial', 
        size=12, 
        color='#525252'
    ),
    annotations = [dict(
        x=0.5,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://en.wikipedia.org/wiki/List_of_deadliest_floods">Wikipedia</a> and <a href="https://countrycode.org">CountryCode</a>',
        showarrow = False
    )]
)

fig.show()